In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
import psycopg2
from pathlib import Path
from collections import Counter
# from config import password

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [5]:
sentiments_data_to_load = "cleaned_sentiment.csv"
sentiments_df = pd.read_csv(sentiments_data_to_load, on_bad_lines='skip')
sentiments_df = sentiments_df.drop(['Unnamed: 0','date','ticker'],axis=1)
sentiments_df.head()

,overall_sent_comp,overall_sent_pos,overall_sent_neg,mean_t_comp_score,mean_t_pos_score,mean_t_neg_score,mean_tgt_comp_score,mean_tgt_pos_score,mean_tgt_neg_score,verb_tense,...,CloseOver60,VolumeOver60,CloseOver90,VolumeOver90,Co30_d,Co60_d,Co90_d,Vo30_d,Vo60_d,Vo90_d
0,0.5078,0.102,0.075,0.08706666666666667,0.11783333333333333,0.0485,-0.5818,0.084,0.157,present,...,127.534833,8.716522e+06,135.033889,10756920.0,0,-1,1,-1,-1,-1
1,0.8658,0.245,0.036,0.1761,0.16466666666666666,0.074,0.7783,0.494,0,present,...,127.534833,8.716522e+06,135.033889,10756920.0,0,-1,1,-1,-1,-1
2,0.5078,0.102,0.075,0.08706666666666667,0.11783333333333333,0.0485,-0.5818,0.084,0.157,present,...,127.534833,8.716522e+06,135.033889,10756920.0,0,-1,1,-1,-1,-1
3,0.5078,0.102,0.075,0.08706666666666667,0.11783333333333333,0.0485,-0.5818,0.084,0.157,present,...,127.534833,8.716522e+06,135.033889,10756920.0,0,-1,1,-1,-1,-1
4,-0.5994,0,0.438,-0.5994,0,0.438,-0.5994,0,0.438,present,...,127.534833,8.716522e+06,135.033889,10756920.0,0,-1,1,-1,-1,-1


In [6]:
sentiments_df = sentiments_df.drop(["overall_sent_comp", "overall_sent_pos",
    "overall_sent_neg", "mean_t_comp_score", "mean_t_pos_score", "mean_t_neg_score", "mean_tgt_comp_score",
    "mean_tgt_pos_score", "mean_tgt_neg_score", "verb_tense"],axis=1)

In [8]:
history_data_to_load = "stock_history.csv"
history_df = pd.read_csv(history_data_to_load, low_memory=False)
history_df.head()

,date,open,high,low,close,volume,ticker
0,1/2/2018 0:00,15.691743,15.980067,15.534477,15.953856,2832700,GME
1,1/3/2018 0:00,15.980067,16.049963,15.656795,15.901433,3789200,GME
2,1/4/2018 0:00,15.901433,16.058699,15.691743,16.006277,2781300,GME
3,1/5/2018 0:00,16.058702,16.364499,15.918909,16.320814,3019000,GME
4,1/8/2018 0:00,16.425656,16.949880,16.425656,16.801350,3668400,GME


In [11]:
history_df['date'] = pd.to_datetime(history_df['date'])
history_df = history_df.drop(['date','ticker'],axis=1)
history_df.head()

,open,high,low,close,volume
0,15.691743,15.980067,15.534477,15.953856,2832700
1,15.980067,16.049963,15.656795,15.901433,3789200
2,15.901433,16.058699,15.691743,16.006277,2781300
3,16.058702,16.364499,15.918909,16.320814,3019000
4,16.425656,16.949880,16.425656,16.801350,3668400


In [14]:
# Combine the data into a single dataset.
sentiments_history_df = pd.merge(sentiments_df, history_df, on=["volume"])
sentiments_history_df.head()

KeyError: 'volume'

In [ ]:
sentiments_history_df = sentiments_df.loc[sentiments_df["ticker"].isin (ticker_list)]

sentiments_history_df.head()

In [ ]:
sentiments_history_df.shape

In [ ]:
X = sentiments_history_df.drop(columns=["mean_custom_score"])
X.shape
y = sentiments_history_df.loc[:, ["mean_custom_score"]].copy()
y.shape

In [ ]:
# Check the balance of our target values
X['ticker'].value_counts()

In [ ]:
from sklearn import preprocessing

def encode_feature(array):
    """ Encode a categorical array into a number array
    
    :param array: array to be encoded
    :return: numerical array
    """
  
    encoder = preprocessing.LabelEncoder()
    encoder.fit(array)
    return encoder.transform(array)

In [ ]:
X["ticker"] = encode_feature(X["ticker"])

In [ ]:
X['ticker'].value_counts()

# MSFT    4
# AAPL    0
# AMZN    1
# FB      2
# GOOG    3

In [ ]:
X.head()

In [ ]:
# Generate our categorical variable lists
sentiments_cat = sentiments_history_df.dtypes[sentiments_history_df.dtypes == "object"].index.tolist()
sentiments_cat

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

In [ ]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

In [ ]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

In [ ]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

In [ ]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

In [ ]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

In [ ]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

In [ ]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

In [ ]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)
from collections import Counter
Counter(y_resampled)

In [ ]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

In [ ]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

In [ ]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

In [ ]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

In [ ]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

In [ ]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

In [ ]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
brf.fit(X_train, y_train)

In [ ]:
# Calculated the balanced accuracy score
y_pred = brf.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

In [ ]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

In [ ]:
# List the features sorted in descending order by feature importance
importances = brf.feature_importances_
cols = X.columns
feature_importances_df = pd.DataFrame({'feature':cols, 'importance': importances})
feature_importances_df.head()
feature_importances_df.sort_values('importance', ascending=False)

In [ ]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier
eec = EasyEnsembleClassifier(n_estimators=100, random_state=1)
eec.fit(X_train, y_train)

In [ ]:
# Calculated the balanced accuracy score
y_pred = eec.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

In [ ]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder

In [ ]:
# Look at APPLICATION_TYPE value counts for binning
comment_counts = sentiments_df["body"].value_counts()
print(len(comment_counts))
comment_counts

In [ ]:
# Visualize the value counts of comments
comment_counts.plot.density()

In [ ]:
# Determine which values to replace if counts are less than ...?
#replace_application = list(application_counts[application_counts < 500].index)

# Replace in dataframe
#for app in replace_application:
    #application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
sentiments_df.body.value_counts()

In [ ]:
# Look at CLASSIFICATION value counts for binning
class_counts = sentiments_df.CLASSIFICATION.value_counts()
class_counts

In [ ]:
# Visualize the value counts of CLASSIFICATION
class_counts.plot.density()

In [ ]:
# Generate our categorical variable lists
comments_cat = sentiments_df.dtypes[sentiments_df.dtypes == "object"].index.tolist()
comments_cat

In [ ]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(sentiments_df[comments_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(sentiments_cat)
encode_df.head()

In [ ]:
# Merge one-hot encoded features and drop the originals
sentiments_df = sentiments_df.merge(encode_df,left_index=True, right_index=True)
sentiments_df = sentiments_df.drop(comments_cat,1)
comments_df.head()

In [ ]:
# Split our preprocessed data into our features and target arrays
y = comments_df["Positive"].values
X = comments_df.drop(["Negative"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

In [ ]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint
# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=1000)

In [ ]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback])

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")